### 프로젝트 종합실습
##### Requests 혹은 Selenium 모듈을 이용해 부동산114 홈페이지로 접속한 다음, 원하는 지역의 아파트 매물 시세 100건을 검색하여 주피터 노트북 화면에 출력하거나 엑셀로 저장해 보세요.
##### 링크 : https://www.r114.com/

### 1. Requests로 크롤링하기

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#아래에 코드 작성
results = []
url = "https://www.r114.com/search/detailSearch.asp"
for page_no in range(1, 20):
    form_data = {
        "dqSearchTerm": "포항",
        "dqSearchTermOld": "포항",
        "dqSearchType": "memul",
        "dqOption": "apt",
        "dqTotalCnt": "",
        "dqSearchSubType": "매매",
        "dqPage": page_no
    }
    res = requests.post(url, data=form_data, verify = False)
    #response 헤더의 Content-Type에 charset 정보가 없음. 이로 인해 response된 데이터가 ISO-8859-1으로
    #인코딩된 것으로 requests 모듈이 자동판단, res.text 값을 ISO-8859-1로 디코딩해버림
    #하지만 실제로는 UTF-8로 인코딩되어 있으므로 UTF-8로 디코딩해야 한글이 정상 출력됨
    #print(res.encoding) # -> 'ISO-8859-1' 출력됨
    html_text = res.content.decode("UTF-8")
    soup = BeautifulSoup(html_text, "html.parser")
    items = soup.select(".list_article > li")
    for item in items:
        size_list = item.select(".txt")[0].text.split(" | ")
        size = [x for x in size_list if "㎡" in x][0]
        this_result_dict = {
            "아파트명" : item.select(".tit_a > span")[0].text,
            "지역" : item.select(".txt > strong")[0].text,
            "크기" : size,
            "매매가" : item.select(".tit_a > em")[0].text.replace("매매", "")
        }
        results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
display(df)

### 2. Selenium으로 크롤링하기

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

#아래에 코드 작성
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.r114.com/")
driver.find_element(By.CSS_SELECTOR, "#dqSearchTerm").send_keys("포항")
driver.find_element(By.CSS_SELECTOR, "#cmdSearchHome").click()
driver.find_element(By.CSS_SELECTOR, ".list_tab_sale > .apt > a").click()
time.sleep(5)
results = []
for i in range(1, 11):
    html_text = driver.page_source
    soup = BeautifulSoup(html_text, "html.parser")
    items = soup.select(".list_article > li")
    for item in items:
        size_list = item.select(".txt")[0].text.split(" | ")
        size = [x for x in size_list if "㎡" in x][0]
        this_result_dict = {
            "아파트명" : item.select(".tit_a > span")[0].text,
            "지역" : item.select(".txt > strong")[0].text,
            "크기" : size,
            "매매가" : item.select(".tit_a > em")[0].text.replace("매매", "")
        }
        results.append(this_result_dict)
    if i < 10:
        for page_no in driver.find_elements(By.CSS_SELECTOR, ".paging > a"):
            if page_no.text.strip() == str(i+1):
                page_no.click()
                break
    
df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
display(df)